Step1: Bad Character 初始化

In [40]:
def generateBC(search_mode, size):
    """Generate the bad character hash table
    
    search_mode: the searching mode
    size: the size of the hash table
    """
    res = [-1] * size # Initialize (-1 because we want to set Xi to -1 if the bad char is not presented in the search mode)
    for idx, char in enumerate(search_mode):
        res[ord(char)] = idx # Set index for the character
    return res

Step2: BM的大框架

- 实现坏字符规则
- 不考虑好后缀规则
- 不考虑坏字符规则向前挪动的问题

In [41]:
def bmSearch(main_mode, search_mode):
    SIZE = 256
    n, m = len(main_mode), len(search_mode)
    
    if n <= m:
        return 0 if main_mode == search_mode else -1
    
    bcList = generateBC(search_mode, SIZE)
    
    i = 0
    while i <= n - m:
        j = m - 1
        while j >= 0:
            if main_mode[i + j] != search_mode[j]:
                break
            else:
                j -= 1
        if j < 0:
            return i
        
        i = i + (j - bcList[ord(main_mode[i + j])]) # here is why we set bcList to -1 in: res = [-1] * size
    
    return -1

Step3: Good Suffix 初始化

拿下标从0到i的子串（i可以是0到m-2）与整个模式串求公共后缀子串

In [25]:
def generateGS(search_mode):
    m = len(search_mode)
    
    suffix = [-1] * m
    prefix = [False] * m
    
    for i in range(0, m - 1): # i从前往后挪动，最远挪动到 m-2
        k = 0
        
        for j in range(i, -1, -1): # j从i往前挪动，最远挪动到 0
            if search_mode[j] == search_mode[m - 1 - k]: # 随着j向前挪动，模式串从最后一位开始往前依次进行比较
                k += 1 # 如果两个字符相等，则长度 k + 1
                suffix[k] = j # 同时更新suffix数组
                
                if j == 0:
                    prefix[k] = True
            else: # 由于j 和 m - 1 - k 是本轮搜索的起始，因此如果不相等说明后续的比较都没有用，直接 break
                break

    return suffix, prefix

Step4: 根据好后缀规则计算模式串的滑动位数

In [35]:
def moveGS(j, m, suffix, prefix):
    """Calculate the move steps using good suffix
    
    j: idx of bad char in search_mode
    m: length of search_mode
    suffix:
    prefix:
    """
    k = m - j - 1 # k is the length of the good suffix
    
    if suffix[k] != -1:
        return j - suffix[k] + 1
    else:
        for r in range(j + 2, m):
            if prefix[m - r]:
                return r # 后缀子串同样也是模式串的前缀子串，因此模式串从0位挪动到 r
        return m

Step5: 整合好后缀规则

In [43]:
def bmSearch(main_mode, search_mode):
    SIZE = 256
    n, m = len(main_mode), len(search_mode)
    
    if n <= m:
        return 0 if main_mode == search_mode else -1
    
    bcList = generateBC(search_mode, SIZE)
    
    suffix, prefix = generateGS(search_mode)
    
    i = 0
    while i <= n - m:
        j = m - 1
        while j >= 0:
            if main_mode[i + j] != search_mode[j]:
                break
            else:
                j -= 1
        if j < 0:
            return i
        
        bc_move = j - bcList[ord(main_mode[i + j])]
        gs_move = 0
        
        if j < m - 1:
            gs_move = moveGS(j, m, suffix, prefix)
        
        i = i + max(bc_move, gs_move)
    
    return -1